## Option Chain Processing with YF 

This notebook demonstrates the following workflow: 
1. Fetch option chain from yf
2. Explore the data structure  
3. Visualize the prices, volumes, and open interests
4. Caclulate basic Greeks
5. Analyze implied vol

For this particular example we use S&P500 and AAPL

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sys.path.append('../src')
from datetime import datetime, timedelta
import plotly.graph_objects as go

pd.options.display.float_format = "{:,.4f}".format
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (14, 6)

# Import our custom pricing functions
import sys
sys.path.append('../src')
from options_desk.pricing.black_scholes import (
    black_scholes_price,
    black_scholes_delta,
    black_scholes_gamma,
    black_scholes_vega,
    black_scholes_theta
)
from options_desk.core.option import OptionType

In [ ]:
spy = yf.Ticker('SPY')
stock_prices = spy.history(period='5d')
current_price = stock_prices['Close'].iloc[-1]

stock_prices[['Close', 'Volume']].tail()

Let's inspect the available expiration dates, which essentially are made by exchanges. 

In [ ]:
spy_options = spy.options

print(f"Available expiration dates ({len(spy_options)} total):")
print("\nFirst 10 expiration dates:")
for i, date in enumerate(spy_options[:10], 1):
    exp_date = pd.to_datetime(date)
    days_to_exp = (exp_date - pd.Timestamp.now()).days_in_month
    print('{} days'.format(days_to_exp))